# CARICAMENTO MODELLO

In [1]:
import torch
import torchvision
import torchvision
from functools import partial
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_the_model(num_classes = 2):
    """
        Metodo per la creazione del modello

        :Params:
            : num_classes = rappresenta il numero di classi di output desiderate. 
    """
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
    return model

num_classes = 2 #[Palla e Background]
model = get_the_model(num_classes)

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Device -> {device}")

model.to(device)

checkpoint_path = '/kaggle/input/result-faster-r-cnn/FASTER-R-CNN_finetuned_epoch-5.pth'

checkpoint = torch.load(checkpoint_path, map_location=torch.device(device))

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epoch = checkpoint['epoch']
losses = checkpoint['loss']

print(f"Ripristinato il modello dopo {epoch+1} epoche, con una perdita di {losses}.")
print(f"Nota che la loss è relativa all'ultimo batch dell'epoca e non è quella media.")

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:00<00:00, 215MB/s]


Device -> cuda


<ipython-input-1-c851faf2f844>:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device(device))


Ripristinato il modello dopo 5 epoche, con una perdita di 0.00832930114120245.
Nota che la loss è relativa all'ultimo batch dell'epoca e non è quella media.


## VISUALIZZAZIONE DI TEST PER UNA SINGOLA IMMAGINE

In [2]:
import os
import random
import torch
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms, models
from torchvision.io import read_image
from torchvision import transforms

if not torch.cuda.is_available():
    image_folder = '/kaggle/input/computer-vision-futbal/dataset/dataset/test/ID-5'
    
    model.eval()
    
    def load_image(image_folder, image_path=None):
        """
            Funzione per caricare una immagine, se image_path è nulla allora estrae un immagine random dalla cartella image_folder.
        """
        if image_path is None:
            image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
            random_image_file = random.choice(image_files)
            image_path = os.path.join(image_folder, random_image_file)
    
        image = Image.open(image_path)
        return image, image_path
    
    def show_image(image):
        image = Image.open(image).convert('RGB')
        plt.imshow(image)
        plt.axis('off')
        plt.show()
    
    image_path  = '/kaggle/input/computer-vision-futbal/dataset/dataset/test/ID-5/frame_2966.jpg'
    
    image, image_path = load_image(image_folder, image_path)
    
    #Stesse trasformazioni in fase di training
    transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = transform(image)
    
    print(f"Visualizzazzione dell'immagine: {image_path}")
    show_image(image_path)
    
    # Aggiunto una dimensione per il batch: [1, C, H, W]
    image_tensor = image.unsqueeze(0)
    
    with torch.no_grad():
        prediction = model(image_tensor)
    
    boxes = prediction[0]['boxes']
    labels = prediction[0]['labels']
    scores = prediction[0]['scores']
    
    print("Risultati dell'inferenza:")
    for i in range(len(boxes)):
        print(f"Box {i}: {boxes[i].tolist()}")
        print(f"Label {i}: {labels[i].item()} (probabilità: {scores[i].item():.4f})")
    
    fig, ax = plt.subplots(1)
    image_pil = Image.open(image_path).convert('RGB')
    ax.imshow(image_pil)
    
    for i, box in enumerate(boxes):
        if scores[i] > 0.5:
            xmin, ymin, xmax, ymax = box
            rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                 linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
    
    plt.axis('off')
    plt.show()
else:
    print("Skipping perchè siamo su GPU...")

Skipping perchè siamo su GPU...


# ANNOTAZIONI JSON

In [3]:
import os
import random
import json
import torch
from PIL import Image
from torchvision import transforms

def process_images(model, image_folder, output_file):

    """
       Per ogni immagine in image_folder esegue in inferenza e salva le previsioni.
       
    :Params:
        _model = Il modello usato;
        :image_folder = La cartella contenente le immagini da elaborare;
        :output_file = Il percorso del file dove salvare i risultati in formato JSON.
    """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando device: {device}")

    model = model.to(device)
    model.eval()

    def load_image(image_folder, image_path=None):
        """
            Funzione per caricare una immagine, se image_path è nulla allora estrae un immagine random dalla cartella image_folder.
        """
        if image_path is None: 
            image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
            random_image_file = random.choice(image_files)
            image_path = os.path.join(image_folder, random_image_file)

        image = Image.open(image_path)
        return image, image_path

    def preprocess_image(image):
        """
            Metodo per richiamare le trasformazioni per rendere l'immagine a come è stata addestrata.
        """
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
        ])
        
        return transform(image)

    results = {}

    imgss = sorted([f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])

    for idx, image_file in enumerate(imgss):
        parti = image_file.split("_")
        numero_str = parti[1].split(".")[0]
        index = int(numero_str)
        
        print(f"Processing file: {image_file} with index:{index}")
        
        image_path = os.path.join(image_folder, image_file)
        image, _ = load_image(image_folder, image_path)
        image_tensor = preprocess_image(image).unsqueeze(0).to(device)
        
        with torch.no_grad():
            prediction = model(image_tensor)
        
        image_name = f"{index:05d}.jpg" 
        bboxes = []
        for box, score in zip(prediction[0]['boxes'], prediction[0]['scores']):
            bboxes.append({
                "bbox": box.tolist(),
                "score": score.item()
            })
        
        results[image_name] = bboxes

    with open(output_file, 'w') as f:
        json.dump(results, f, indent=4)

    print(f"Results saved in: {output_file}")

In [4]:
image_folder = '/kaggle/input/computer-vision-futbal/dataset/dataset/test/ID-5'
output_file = '/kaggle/working/bounding_boxes_5_model.json'
process_images(model, image_folder, output_file)

Usando device: cuda
Processing file: frame_0.jpg with index:0
Processing file: frame_1.jpg with index:1
Processing file: frame_10.jpg with index:10
Processing file: frame_100.jpg with index:100
Processing file: frame_1000.jpg with index:1000
Processing file: frame_1001.jpg with index:1001
Processing file: frame_1002.jpg with index:1002
Processing file: frame_1003.jpg with index:1003
Processing file: frame_1004.jpg with index:1004
Processing file: frame_1005.jpg with index:1005
Processing file: frame_1006.jpg with index:1006
Processing file: frame_1007.jpg with index:1007
Processing file: frame_1008.jpg with index:1008
Processing file: frame_1009.jpg with index:1009
Processing file: frame_101.jpg with index:101
Processing file: frame_1010.jpg with index:1010
Processing file: frame_1011.jpg with index:1011
Processing file: frame_1012.jpg with index:1012
Processing file: frame_1013.jpg with index:1013
Processing file: frame_1014.jpg with index:1014
Processing file: frame_1015.jpg with ind

In [5]:
image_folder = '/kaggle/input/computer-vision-futbal/dataset/dataset/test/ID-6'
output_file = '/kaggle/working/bounding_boxes_6_model.json'
process_images(model, image_folder, output_file)

Usando device: cuda
Processing file: frame_0.jpg with index:0
Processing file: frame_1.jpg with index:1
Processing file: frame_10.jpg with index:10
Processing file: frame_100.jpg with index:100
Processing file: frame_1000.jpg with index:1000
Processing file: frame_1001.jpg with index:1001
Processing file: frame_1002.jpg with index:1002
Processing file: frame_1003.jpg with index:1003
Processing file: frame_1004.jpg with index:1004
Processing file: frame_1005.jpg with index:1005
Processing file: frame_1006.jpg with index:1006
Processing file: frame_1007.jpg with index:1007
Processing file: frame_1008.jpg with index:1008
Processing file: frame_1009.jpg with index:1009
Processing file: frame_101.jpg with index:101
Processing file: frame_1010.jpg with index:1010
Processing file: frame_1011.jpg with index:1011
Processing file: frame_1012.jpg with index:1012
Processing file: frame_1013.jpg with index:1013
Processing file: frame_1014.jpg with index:1014
Processing file: frame_1015.jpg with ind

# POST PROCESS

In [6]:
PATH_OUT = "/kaggle/working"
PATH_XML_5 = "/kaggle/input/computer-vision-futbal/dataset_soccer/dataset_soccer/test/ID-5.xml"
PATH_XML_6 = "/kaggle/input/computer-vision-futbal/dataset_soccer/dataset_soccer/test/ID-6.xml"
#PATH_5 = "/kaggle/input/faster-json-w-bounding-boxes/bounding_boxes_5.json"
#PATH_6 = "/kaggle/input/faster-json-w-bounding-boxes/bounding_boxes_6.json"
PATH_5 = '/kaggle/working/bounding_boxes_5_model.json'
PATH_6 = '/kaggle/working/bounding_boxes_6_model.json'

## METODI AUSILIARI

In [7]:
import os
import re
import json
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

plt.rcParams["figure.figsize"] = (10, 8)

In [8]:
class TrackingEvaluator:
    def __init__(self, gt_ann_file, pred_file):
        self.gt_ann_file = gt_ann_file
        self.pred_file = pred_file
        self.gt_data_points = {}
        self.pred_points = {}
        self.frames_idx = None

    @staticmethod
    def extract_points_data(xml_content):
        root = ET.fromstring(xml_content)
        points_data = {}
        
        for track in root.findall(".//track"):
            for point in track.findall("points"):
                data = {
                    'frame': int(point.get("frame")),
                    'outside': int(point.get("outside")),
                    'occluded': int(point.get("occluded")),
                    'keyframe': int(point.get("keyframe")),
                    'points': tuple(map(float, point.get("points").split(","))),
                    'z_order': int(point.get("z_order")),
                }
                if data['frame'] in points_data:
                    print(f'Alert: multiple frame entries for ID {data["frame"]}')
                points_data[data['frame']] = data
        
        return points_data

    @staticmethod
    def _convert_key(k):
        return int(Path(k).stem)

    def load_data(self):
        # Load ground truth data
        gt_content = Path(self.gt_ann_file).read_text()
        self.gt_data_points = self.extract_points_data(gt_content)

        # Load prediction data
        pred_content = Path(self.pred_file).read_text().replace('-Infinity', '-1').replace('Infinity', '-1')
        raw_pred_points = json.loads(pred_content)
        self.pred_points = {
            self._convert_key(k): v for k, v in raw_pred_points.items() if v['x'] >= 0
        }

    def compute_frame_indices(self):
        ordered_list_pred_frame = sorted(self.pred_points.keys())
        ordered_list_gt_frame = sorted(self.gt_data_points.keys())
        
        print(f'GT   frames: {ordered_list_gt_frame[0]} - {ordered_list_gt_frame[-1]}')
        print(f'PRED frames: {ordered_list_pred_frame[0]} - {ordered_list_pred_frame[-1]}')

        self.frames_idx = (
            min(ordered_list_gt_frame[0], ordered_list_pred_frame[0]),
            max(ordered_list_gt_frame[-1], ordered_list_pred_frame[-1]),
        )
        print(f'Frame Index Range: {self.frames_idx}')

    @staticmethod
    def is_match(x1, y1, x2, y2, threshold=4):
        p1 = np.array((x1, y1))
        p2 = np.array((x2, y2))
        euclid_dist = np.sqrt(np.dot((p1 - p2).T, (p1 - p2)))
        return euclid_dist < threshold

    def evaluate_metrics(self):
        cnt_match = 0
        cnt_no_match = 0
        cnt_no_pred = 0
        cnt_no_frame = 0

        for i in range(self.frames_idx[0], self.frames_idx[1] + 1):
            if i not in self.gt_data_points:
                cnt_no_frame += 1
                continue
            if i not in self.pred_points:
                cnt_no_pred += 1
                continue

            p1 = self.gt_data_points[i]
            p2 = self.pred_points[i]

            if self.is_match(*p1['points'], p2['x'], p2['y']):
                cnt_match += 1
            else:
                cnt_no_match += 1

        total_frames = len(self.gt_data_points)
        print(f'Total frames: {total_frames}')
        print(f'Total predictions: {len(self.pred_points)}')
        print(f'Matches: {cnt_match} ({cnt_match / total_frames:.3f})')
        print(f'No matches: {cnt_no_match} ({cnt_no_match / total_frames:.3f})')
        print(f'No predictions: {cnt_no_pred} ({cnt_no_pred / total_frames:.3f})')
        print(f'No frame data: {cnt_no_frame} ({cnt_no_frame / (self.frames_idx[1] - self.frames_idx[0] + 1):.3f})')

    def compute_tracking_sequence(self):
        norm_width = 1920
        norm_height = 1080

        gt_seq = []
        pred_seq = []

        for i in range(min(self.gt_data_points.keys()), max(self.gt_data_points.keys()) + 1):
            if i in self.gt_data_points:
                if i not in self.pred_points:
                    pred_seq.append((0, 0))
                else:
                    p2 = self.pred_points[i]
                    pred_seq.append((p2['x'] / norm_width, p2['y'] / norm_height))

                x, y = self.gt_data_points[i]['points']
                gt_seq.append((x / norm_width, y / norm_height))

        return gt_seq, pred_seq

    def compute_mse(self):
        gt_seq, pred_seq = self.compute_tracking_sequence()
        mse = mean_squared_error(gt_seq, pred_seq)
        print(f'Mean Squared Error: {mse}')
        return mse

In [9]:
def open_json(path):
    """
        Il metodo si occupa di caricare un file JSON.
    
        :Params:
            :path = Il percorso del file JSON da aprire.
    
        :Returns:
            :data_result = E' il dizionario con gli elementi ordinati in base alla 
                parte numerica della chiave. Deve essere ordinato poichè il JSON possiede le chiavi
                nel seguente modo: "00000.jpg", seguento l'ordinamento lessicale si ottiene che la 
                chiave "000010.jpg" viene prima di "00002.jpg" e questo rompe l'ordinamento del video.
    """
    with open(path, 'r') as file:
        data = json.load(file)
    
    ordinato = dict(sorted(data.items(), key=lambda x: int(x[0].split('.')[0])))
    
    return ordinato

def save_json(path, data):
    """
        Il metodo si occupa di salvare il file JSON in una specifica posizione.

        :Params:
            :path = Indica il percorso in cui salvare il file JSON;
            :data = Rappresenta il file da salvare.
    """
    
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)
        
    print(f"JSON saved in: {path}")

# CALCOLO MSE

## ALTRI METODI AUSILIARI

In [10]:
import json

def filter_and_transform_bboxes(data, score_threshold=0.1):
    """    
        Questa funzione filtra e trasforma le bounding boxes per ogni frame, presenti
        nel file fornito in input, selezionando solo quelle con un punteggio maggiore 
        o uguale alla soglia specificata. 
        Per ogni frame, dopo aver selezionato le bounding box con score maggiore
        del valore di soglia, viene selezionata la bounding box con il punteggio 
        massimo e trasformata nel formato richiesto, contenente le coordinate della 
        bounding box (xmin, ymin, xmax, ymax) e il relativo score di predizione.

        :Params:
            :data = Il file JSON delle predizioni che ha come chiave "00000.jgp" e una 
                lista di bounding boxes come valore. 
                Ogni bounding box è un dizionario che contiene le coordinate e lo score di 
                predizione.
            :score_threshold = Rappresenta la soglia del punteggio minimo per considerare 
            una bounding box.

        :Return:
            :result = Un file JSON dove per ogni frame vengono contenute le informazioni filtrate.
    """
    result = {}
    
    for file_name in sorted(data.keys()):
        bboxes = data[file_name]
        
        if bboxes:

            filtered_bboxes = [bbox for bbox in bboxes if bbox['score'] >= score_threshold]
            
            if filtered_bboxes:
                
                #La migliore bb è quella che possiede lo score maggiore.
                best_bbox = max(filtered_bboxes, key=lambda x: x['score'])
                
                result[file_name] = {
                    'xmin': best_bbox['bbox'][0],
                    'ymin': best_bbox['bbox'][1],
                    'xmax': best_bbox['bbox'][2],
                    'ymax': best_bbox['bbox'][3],
                    'score': best_bbox['score']
                }
            else:
                result[file_name] = {}
        else:
            result[file_name] = {}
    
    return result

In [11]:
def rimuovi_falsi_positivi(data):
    """    
        Questa funzione analizza i frame contenuti nel file JSON delle predizioni, 
        e rimuove i frame che vengono considerati falsi positivi.
        Un frame viene considerato un falso positivo se soddisfa due condizioni:
            - Se il punteggio di predizione è basso (minore di una certa soglia).
            - Se l'immagine ha molti frame vuoti sia precedenti che successivi, 
                 suggerendo che il rilevamento potrebbe essere errato o irrilevante.
        
        Per determinare se un frame è un falso positivo, la funzione verifica i frame 
        precedenti e successivi per determinare se sono vuoti. 
        Se ci sono almeno 5 frame vuoti prima e dopo e il punteggio è basso, 
        l'immagine viene contrassegnata come un falso positivo e il suo contenuto 
        viene rimosso (impostato a un dizionario vuoto).
        
        :Params:
            :data = Un file JSON contenente le predizioni, con il nome del frame come chiave 
                    e un dizionario che rappresenta le coordinate della bounding box e 
                    il punteggio di predizione come valore.
        :Return:
            :data = Il file JSON delle predizioni aggiornato, dove le immagini considerate 
                    falsi positivi sono state rimosse.
    """
    
    images = list(data.keys())

    for i, image in enumerate(images):

        if data[image]:
            score = data[image].get('score', 0)

            prev_empty_count = 0
            next_empty_count = 0

            # Prendiamoci quanti frame precedenti e successivi sono vuoti
            for j in range(i-1, -1, -1):
                if not data[images[j]]:
                    prev_empty_count += 1
                else:
                    break 

            for j in range(i + 1, len(images)):
                if not data[images[j]]: 
                    next_empty_count += 1
                else:
                    break 

           #Condizione per considerarlo falso positivo:
                #almeno 5 frame prima e dopo nulli e score < 0.99
                #score < 0.7
            if (prev_empty_count >=5  and next_empty_count >= 5 and score < 0.99) or score < 0.70:
                print(f"Suspicious frame: {image}, with {prev_empty_count} empty frames before and {next_empty_count} empty frames after, info: {data[image]}")
                data[image] = {}
    return data


In [12]:
import json

def process_bounding_boxes(input_data):
    """    
        Questa funzione elabora le informazioni delle bounding boxes per ogni frame nel 
        file JSON di input, calcolando il centro di ogni bounding box e restituendo un 
        nuovo file JSON con le coordinate (x, y) dei centri, come richiesto.
            
        :Params:
            :input_data = Un file JSON contenente le informazioni delle bounding boxes, 
                        dove il nome del frame è la chiave e il valore è un dizionario 
                        con le coordinate xmin, ymin, xmax, ymax della bounding box.
        
        :Return:
            :output_data = Un file JSON contenente per ogni frame il centro della 
                           bounding box sotto forma di coordinate (x, y). Se non 
                           ci sono dati per un frame, viene restituito (-1, -1) come coordinate.
    """
    output_data = {}

    for filename, bbox_info in input_data.items():
        if bbox_info:
            
            xmin = bbox_info.get('xmin')
            ymin = bbox_info.get('ymin')
            xmax = bbox_info.get('xmax')
            ymax = bbox_info.get('ymax')

            center_x = (xmin + xmax) / 2
            center_y = (ymin + ymax) / 2
            
            output_data[filename] = {'x': center_x, 'y': center_y}
        else:
            output_data[filename] = {'x': -1, 'y': -1}
    
    return output_data

## CALCOLO MSE SENZA POST PROCESSING

In [13]:
save_json(f"{PATH_OUT}/json_5_nots.json",process_bounding_boxes(filter_and_transform_bboxes(open_json(PATH_5),score_threshold=0)))

JSON saved in: /kaggle/working/json_5_nots.json


In [14]:
save_json(f"{PATH_OUT}/json_6_nots.json",process_bounding_boxes(filter_and_transform_bboxes(open_json(PATH_6),score_threshold=0)))

JSON saved in: /kaggle/working/json_6_nots.json


In [15]:
gt_ann_file = Path.home() / PATH_XML_5
pred_file = Path.home() / f'{PATH_OUT}/json_5_nots.json'

evaluator = TrackingEvaluator(gt_ann_file, pred_file)
evaluator.load_data()
evaluator.compute_frame_indices()
evaluator.evaluate_metrics()
mse5 = evaluator.compute_mse(); 

GT   frames: 420 - 2580
PRED frames: 32 - 2966
Frame Index Range: (32, 2966)
Total frames: 346
Total predictions: 534
Matches: 265 (0.766)
No matches: 27 (0.078)
No predictions: 54 (0.156)
No frame data: 2589 (0.882)
Mean Squared Error: 0.05456320838758867


In [16]:
gt_ann_file = Path.home() / PATH_XML_6
pred_file = Path.home() / f'{PATH_OUT}/json_6_nots.json'

evaluator = TrackingEvaluator(gt_ann_file, pred_file)
evaluator.load_data()
evaluator.compute_frame_indices()
evaluator.evaluate_metrics()
mse6 = evaluator.compute_mse(); 

GT   frames: 417 - 2495
PRED frames: 0 - 2962
Frame Index Range: (0, 2962)
Total frames: 263
Total predictions: 981
Matches: 246 (0.935)
No matches: 9 (0.034)
No predictions: 8 (0.030)
No frame data: 2700 (0.911)
Mean Squared Error: 0.018005002783646654


In [17]:
print(f'FINAL MSE {np.mean([mse5, mse6])}')

FINAL MSE 0.03628410558561766


## CALCOLO MSE CON TRESHOLDING

In [18]:
save_json(f"{PATH_OUT}/json_5_ts.json",process_bounding_boxes(rimuovi_falsi_positivi(filter_and_transform_bboxes(open_json(PATH_5),score_threshold=0.8))))

Suspicious frame: 00032.jpg, with 32 empty frames before and 18 empty frames after, info: {'xmin': 316.5431213378906, 'ymin': 55.160499572753906, 'xmax': 341.571533203125, 'ymax': 80.30467224121094, 'score': 0.904865562915802}
Suspicious frame: 00051.jpg, with 51 empty frames before and 6 empty frames after, info: {'xmin': 277.176025390625, 'ymin': 58.080078125, 'xmax': 302.1291809082031, 'ymax': 83.93463897705078, 'score': 0.8420079946517944}
Suspicious frame: 00058.jpg, with 58 empty frames before and 365 empty frames after, info: {'xmin': 263.1426696777344, 'ymin': 57.018028259277344, 'xmax': 288.32879638671875, 'ymax': 81.7767105102539, 'score': 0.8761492967605591}
Suspicious frame: 00694.jpg, with 75 empty frames before and 34 empty frames after, info: {'xmin': 1905.5531005859375, 'ymin': 124.76521301269531, 'xmax': 1920.0, 'ymax': 150.0313262939453, 'score': 0.8463476896286011}
Suspicious frame: 00729.jpg, with 110 empty frames before and 19 empty frames after, info: {'xmin': 186

In [19]:
save_json(f"{PATH_OUT}/json_6_ts.json",process_bounding_boxes(rimuovi_falsi_positivi(filter_and_transform_bboxes(open_json(PATH_6),score_threshold=0.8))))

Suspicious frame: 00859.jpg, with 218 empty frames before and 641 empty frames after, info: {'xmin': 1901.3675537109375, 'ymin': 977.2528076171875, 'xmax': 1920.0, 'ymax': 1003.6451416015625, 'score': 0.8416836261749268}
Suspicious frame: 01501.jpg, with 860 empty frames before and 13 empty frames after, info: {'xmin': 1628.141357421875, 'ymin': 12.353048324584961, 'xmax': 1654.0811767578125, 'ymax': 38.62401580810547, 'score': 0.8094037771224976}
Suspicious frame: 01596.jpg, with 78 empty frames before and 490 empty frames after, info: {'xmin': 1610.1160888671875, 'ymin': 14.827753067016602, 'xmax': 1636.2947998046875, 'ymax': 41.9946403503418, 'score': 0.8214535713195801}
Suspicious frame: 02087.jpg, with 569 empty frames before and 5 empty frames after, info: {'xmin': 1643.8026123046875, 'ymin': 64.59578704833984, 'xmax': 1669.826171875, 'ymax': 90.81431579589844, 'score': 0.9194740056991577}
Suspicious frame: 02093.jpg, with 575 empty frames before and 352 empty frames after, info:

In [20]:
gt_ann_file = Path.home() / PATH_XML_5
pred_file = Path.home() / f'{PATH_OUT}/json_5_ts.json'

evaluator = TrackingEvaluator(gt_ann_file, pred_file)
evaluator.load_data()
evaluator.compute_frame_indices()
evaluator.evaluate_metrics()
mse5 = evaluator.compute_mse(); 

GT   frames: 420 - 2580
PRED frames: 424 - 2966
Frame Index Range: (420, 2966)
Total frames: 346
Total predictions: 281
Matches: 255 (0.737)
No matches: 10 (0.029)
No predictions: 81 (0.234)
No frame data: 2201 (0.864)
Mean Squared Error: 0.07505830962815216


In [21]:
gt_ann_file = Path.home() / PATH_XML_6
pred_file = Path.home() / f'{PATH_OUT}/json_6_ts.json'

evaluator = TrackingEvaluator(gt_ann_file, pred_file)
evaluator.load_data()
evaluator.compute_frame_indices()
evaluator.evaluate_metrics()
mse6 = evaluator.compute_mse(); 

GT   frames: 417 - 2495
PRED frames: 418 - 2494
Frame Index Range: (417, 2495)
Total frames: 263
Total predictions: 248
Matches: 242 (0.920)
No matches: 3 (0.011)
No predictions: 18 (0.068)
No frame data: 1816 (0.873)
Mean Squared Error: 0.028571952201042798


In [22]:
print(f'FINAL MSE {np.mean([mse5, mse6])}')

FINAL MSE 0.05181513091459748


Le considerazioni che possiamo trarre sono le seguenti:

 **VIDEO 5 SENZA TS**
* Total frames: 346
* Total predictions: 534
* MSE = 0.05456320838758867

 **VIDEO 5 CON TS**
* Total frames: 346
* Total predictions: 281
* MSE = 0.07505830962815216

 **VIDEO 6 SENZA TS**
* Total frames: 346
* Total predictions: 534
* MSE = 0.018005002783646654

 **VIDEO 6 CON TS**
* Total frames: 263
* Total predictions: 248
* MSE = 0.028571952201042798

Notiamo che anche impostando un valore di tresholding pari a 0.8 abbattiamo tanti falsi positivi, lo si può notare dalle total prediction.
Questo risultato è anche ottenuto grazie all'invocazione della semplice funzione di rimozione dei falsi positivi.
Ma tralasciando il conteggio dei falsi positivi (anche se impatta molto nel risultato visivo finale) il valore dell'MSE tende ad aumentare, questo perchè dimunuiscono i valori di No Matched e aumentano quelli di No Prediction.

**media MSE video 5 e 6 senza TS**

FINAL MSE 0.03628410558561766


**media MSE video 5 e 6 con TS**

FINAL MSE 0.05181513091459748

L'ideale sarebbe abbassare il valore dell'MSE dei risultati prodotti con il tresholding in quanto più accurati. Per ridurre il valore dei no prediction usiamo un interpolazione classica, lineare.

## INTERPOLAZIONE

L'interpolazione lineare è un metodo per stimare un valore all'interno di un intervallo, partendo da due punti noti. L'idea è quella di applicarla per colmare un gap di frame mancanti tra due predizioni.

In [23]:
def interpolare_bounding_box(frame_precedente, frame_successivo, dist_prec, dist_succ):
    """    
        Esegue l'interpolazione lineare delle coordinate della bounding box tra due frame adiacenti 
        in base alla distanza tra il frame corrente e ciascuno dei frame validi. 
        
        :Params:
            :frame_precedente = Il dizionario contenente le informazioni della bounding box 
                                 e lo score del frame precedente;
            :frame_successivo = Il dizionario contenente le informazioni della bounding box
                                 e lo score del frame successivo;
            :dist_prec = La distanza tra il frame corrente e il frame precedente:
            :dist_succ = La distanza tra il frame corrente e il frame successivo.
        
        :Return:
            :return = Un dizionario con le coordinate della bounding box interpolata.
    """
    xmin_interpolato = frame_precedente["xmin"]+ (dist_prec / (dist_prec + dist_succ)) * (frame_successivo["xmin"] - frame_precedente["xmin"])
    ymin_interpolato = frame_precedente["ymin"] + (dist_prec / (dist_prec + dist_succ)) * (frame_successivo["ymin"] -frame_precedente["ymin"])
    xmax_interpolato = frame_precedente["xmax"] +(dist_prec / (dist_prec + dist_succ)) * (frame_successivo["xmax"] -frame_precedente["xmax"])
    ymax_interpolato = frame_precedente["ymax"] + (dist_prec / (dist_prec + dist_succ)) * (frame_successivo["ymax"]- frame_precedente["ymax"])

    return {
        "xmin": xmin_interpolato,
        "ymin": ymin_interpolato,
        "xmax": xmax_interpolato,
        "ymax": ymax_interpolato
    }

def interpolare_score(frame_precedente, frame_successivo, dist_prec, dist_succ):
    """    
        Calcola la media pesata degli score tra il frame precedente e successivo, 
        basata sulle distanze tra il frame corrente e i due frame validi. 
        Lo score risultante da un peso maggiore al frame più vicino al frame corrente.
        
        :Params:
            :frame_precedente = Il dizionario contenente lo score del frame precedente;
            :frame_successivo = Il dizionario contenente lo score del frame successivo;
            :dist_prec = La distanza tra il frame corrente e il frame precedente;
            :dist_succ = La distanza tra il frame corrente e il frame successivo.
        
        :Return:
            :score_interpolato = Lo score interpolato risultante dalla media pesata tra i due score.
    """
    score_prec = frame_precedente["score"]
    score_succ = frame_successivo["score"]
    score_interpolato = (score_prec * dist_succ + score_succ * dist_prec) / (dist_prec + dist_succ)
    return score_interpolato

def interpolare_frames(frames, gap=5):
    """    
        Funzione di richiamo per quelle descritte in precedenza. La funzione cerca i frame vuoti 
        e, se sono entro una distanza massima tra i frame validi precedenti e successivi, interpola 
        le loro bounding box e gli score.
        
        L'interpolazione delle bounding boxes avviene tramite interpolazione lineare tra i 
        frame adiacenti, mentre lo score viene calcolato come una media pesata in base alla distanza 
        tra i frame adiacenti e il frame da interpolare. 
        
        :Params:
            :frames = Un dizionario contenente i frame, dove la chiave è il nome del frame e 
                      il valore è un dizionario con le informazioni della 
                      bounding box (xmin, ymin, xmax, ymax) e lo score del frame.
            :gap = La distanza massima tra i frame validi che viene considerata per l'interpolazione. 
                   Se la distanza tra un frame vuoto e i frame validi circostanti è maggiore di questo 
                   valore, non viene eseguita l'interpolazione.
        :Return:
            :frames = Il dizionario aggiornato con le bounding boxes e gli score interpolati per i frame vuoti.
    """
    
    frame_keys = list(frames.keys())
    
    for i in range(1, len(frame_keys) - 1):
        current_frame = frame_keys[i]
    
        if frames[current_frame] == {}:
            previous_frame = None
            next_frame = None

            for j in range(i-1, -1, -1):
                if frames[frame_keys[j]] != {}:
                    previous_frame = frame_keys[j]
                    break

            for k in range(i+1, len(frame_keys)):
                if frames[frame_keys[k]] != {}:
                    next_frame = frame_keys[k]
                    break
            
            if previous_frame and next_frame:
                dist_prec = abs(i - frame_keys.index(previous_frame))  # Dist tra precedente e corrente
                dist_succ = abs(i - frame_keys.index(next_frame))  #Dist tra successivo e corrente
                if dist_prec + dist_succ <= gap:
                    frames[current_frame] = interpolare_bounding_box(frames[previous_frame], frames[next_frame], dist_prec, dist_succ)
                    frames[current_frame]["score"] = interpolare_score(frames[previous_frame], frames[next_frame], dist_prec, dist_succ)
    
    return frames


In [24]:
save_json(f"{PATH_OUT}/json_5_interpolated.json",process_bounding_boxes(interpolare_frames(filter_and_transform_bboxes(open_json(PATH_5),score_threshold=0.5),gap=35)))

JSON saved in: /kaggle/working/json_5_interpolated.json


In [25]:
save_json(f"{PATH_OUT}/json_6_interpolated.json",process_bounding_boxes(interpolare_frames(filter_and_transform_bboxes(open_json(PATH_6),score_threshold=0.5),gap=35)))

JSON saved in: /kaggle/working/json_6_interpolated.json


In [26]:
gt_ann_file = Path.home() / PATH_XML_5
pred_file = Path.home() / f'{PATH_OUT}/json_5_interpolated.json'

evaluator = TrackingEvaluator(gt_ann_file, pred_file)
evaluator.load_data()
evaluator.compute_frame_indices()
evaluator.evaluate_metrics()
mse5 = evaluator.compute_mse(); 

GT   frames: 420 - 2580
PRED frames: 32 - 2966
Frame Index Range: (32, 2966)
Total frames: 346
Total predictions: 537
Matches: 276 (0.798)
No matches: 48 (0.139)
No predictions: 22 (0.064)
No frame data: 2589 (0.882)
Mean Squared Error: 0.024265127309402047


In [27]:
gt_ann_file = Path.home() / PATH_XML_6
pred_file = Path.home() / f'{PATH_OUT}/json_6_interpolated.json'

evaluator = TrackingEvaluator(gt_ann_file, pred_file)
evaluator.load_data()
evaluator.compute_frame_indices()
evaluator.evaluate_metrics()
mse6 = evaluator.compute_mse(); 

GT   frames: 417 - 2495
PRED frames: 7 - 2615
Frame Index Range: (7, 2615)
Total frames: 263
Total predictions: 418
Matches: 249 (0.947)
No matches: 12 (0.046)
No predictions: 2 (0.008)
No frame data: 2346 (0.899)
Mean Squared Error: 0.005526862526945486


In [28]:
print(f'FINAL MSE {np.mean([mse5, mse6])}')

FINAL MSE 0.014895994918173765


Notiamo che il valore di MSE finale è molto basso ma abbiamo la presenza di tanti falsi positivi che impattano nel video finale di output. Di seguito viene presentato l'ultimo metodo di post-process.

In [29]:
def post_process(json_data_original, json_data_ts):
    """
        Funzione main di post-processing per elaborare e correggere i dati delle bounding box.

        L'idea alla base è la seguente:
            @ (1)Prendere il file con tresholding con un valore relativamente basso ed successivamente interpolato;
            @ (2)Prendere il file con tresholding con un valore estremamente alto;
            - Usare il (2) per andare a rilevare i limiti da dove iniziano e finiscono le predizioni.
              Quest'ultime vengono prese solo quelle sicure, ecco perchè lo score è alto.
            - Eliminare tutte i frame dal (1) che stanno al di fuori dell'intervallo individuato in precedenza.
            - Iterare su questo intervallo per controllare se ci sono dei falsi positivi, ossia singoli frame,
              diversi da -1,-1 che sono circondati da tanti -1,-1;
            - Iterare nuovamente su questo intervallo e considerare sequenze di frame diversi da -1,-1 che 
            potrebbero essere dei falsi positivi, ossia una sequenza relativamente piccola che è circondata 
            da -1,-1.
            @ Restituire il file interpolato che ha subito tutte le modifiche necessarie.
            
        :Params:
            :json_data_original: Un dizionario contenente i dati interpolati.
            :json_data_ts: Un dizionario contenente i dati post operazione di tresholding.
            
        :Return:
            :json_data_original: Il dizionario aggiornato con i dati modificati.
    """

    print("Starting Post-Processing...")
    
    index_start = -1
    index_end = -1

    #TROVIAMO INDICE DI INIZIO E FINE
    for i in range(len(json_data_ts)):
        file_name = f"{i:05d}.jpg"
        if json_data_ts[file_name]['x'] != -1 and json_data_ts[file_name]['y'] != -1:
            index_start = i
            break

    for j in range(len(json_data_ts)-1, -1, -1):
        file_name = f"{j:05d}.jpg"
        if json_data_ts[file_name]['x'] != -1 and json_data_ts[file_name]['y'] != -1:
            index_end = j
            break

    #Abbiamo gli indici per individuare le irrilevant prediction
    print(f"Index founded. Index Start:{index_start}, Index End:{index_end}")

    # SCARTIAMO TUTTI I FRAME CHE SONO AL DI FUORI DI QUESTO RANGE / IRRILEVANT ANNOTATION
    for k in range(len(json_data_original)):
        file_name = f"{k:05d}.jpg"
        if k < index_start or k > index_end:
            json_data_original[file_name]['x'] = -1
            json_data_original[file_name]['y'] = -1


    #OPERIAMO SULLE STRONG ANNOTATION DATE DA json_data_ts E SULLE WEAK ANNOTATION 
    #CHE SONO QUELLE CHE RIMANGONO DALLE PRECEDENTI OPERAZIONI.
    
    #ELIMINAZIONE DEI FALSI POSITIVI / WEAK ANNOTATION INUTILI
    for l in range(index_start + 1, index_end):
        count_invalid_before = 0
        count_invalid_after = 0
        frame_name = f"{l:05d}.jpg"

        # Conteggio frame = a -1,-1 pre e dopo quello corrente
        if json_data_original[frame_name]['x'] != -1 and json_data_original[frame_name]['y'] != -1:
            i = l - 1
            while i >= 0 and json_data_original[f"{i:05d}.jpg"]['x'] == -1 and json_data_original[f"{i:05d}.jpg"]['y'] == -1:
                count_invalid_before += 1
                i -= 1

            i = l + 1
            while i <= index_end and json_data_original[f"{i:05d}.jpg"]['x'] == -1 and json_data_original[f"{i:05d}.jpg"]['y'] == -1:
                count_invalid_after += 1
                i += 1

        if count_invalid_after >= 10 and count_invalid_before >= 10:
            json_data_original[frame_name]['x'] = -1
            json_data_original[frame_name]['y'] = -1

    # CONTROLLO DELLA WEAK SEQUENCE
    for l in range(index_start, index_end+1):
        valid_frame_sequence = []
        count_invalid_before_sequence = 0
        count_invalid_after_sequence = 0
        frame_name = f"{l:05d}.jpg"
        
        if json_data_original[frame_name]['x'] != -1 and json_data_original[frame_name]['y'] != -1:

            valid_frame_sequence.append(l)
            
            i = l + 1
            while i <= index_end and json_data_original[f"{i:05d}.jpg"]['x'] != -1 and json_data_original[f"{i:05d}.jpg"]['y'] != -1:
                valid_frame_sequence.append(i)
                i += 1

            #Da qui iniza il controllo della weak sequence
            if 15 <= len(valid_frame_sequence) <= 70:

                i = valid_frame_sequence[0] - 1
                while i >= 0 and json_data_original[f"{i:05d}.jpg"]['x'] == -1 and json_data_original[f"{i:05d}.jpg"]['y'] == -1:
                    count_invalid_before_sequence += 1
                    i -= 1
                
                i = valid_frame_sequence[-1] + 1
                while i <= index_end and json_data_original[f"{i:05d}.jpg"]['x'] == -1 and json_data_original[f"{i:05d}.jpg"]['y'] == -1:
                    count_invalid_after_sequence += 1
                    i += 1
                
                # Se i valori -1,-1 prima e dopo sono abbastanza grandi rimuoviamo la sequenza
                if count_invalid_before_sequence >= 40 and count_invalid_after_sequence >= 40:
                    print(f"Sequence: {valid_frame_sequence} with total invalid frame before:{count_invalid_before_sequence} and total invalid frame after: {count_invalid_after_sequence}")
                    for frame_idx in valid_frame_sequence:
                        frame_name = f"{frame_idx:05d}.jpg"
                        json_data_original[frame_name]['x'] = -1
                        json_data_original[frame_name]['y'] = -1

    print("Post-Processing finished!")
    return json_data_original


In [30]:
save_json(f"{PATH_OUT}/ID_5_256942_252050.json",post_process(open_json(f"{PATH_OUT}/json_5_interpolated.json"),process_bounding_boxes(rimuovi_falsi_positivi(filter_and_transform_bboxes(open_json(PATH_5),score_threshold=0.90)))))

Suspicious frame: 00032.jpg, with 32 empty frames before and 391 empty frames after, info: {'xmin': 316.5431213378906, 'ymin': 55.160499572753906, 'xmax': 341.571533203125, 'ymax': 80.30467224121094, 'score': 0.904865562915802}
Suspicious frame: 00729.jpg, with 110 empty frames before and 19 empty frames after, info: {'xmin': 1861.1156005859375, 'ymin': 78.15499114990234, 'xmax': 1887.348388671875, 'ymax': 104.8904037475586, 'score': 0.9478008151054382}
Suspicious frame: 00749.jpg, with 130 empty frames before and 146 empty frames after, info: {'xmin': 1213.257080078125, 'ymin': 57.7552604675293, 'xmax': 1241.1192626953125, 'ymax': 83.89463806152344, 'score': 0.9620065093040466}
Suspicious frame: 00896.jpg, with 277 empty frames before and 1547 empty frames after, info: {'xmin': 1906.5347900390625, 'ymin': 235.0340118408203, 'xmax': 1920.0, 'ymax': 261.00128173828125, 'score': 0.9795364141464233}
Starting Post-Processing...
Index founded. Index Start:424, Index End:2966
Sequence: [694,

In [31]:
save_json(f"{PATH_OUT}/ID_6_256942_252050.json",post_process(open_json(f"{PATH_OUT}/json_6_interpolated.json"),process_bounding_boxes(rimuovi_falsi_positivi(filter_and_transform_bboxes(open_json(PATH_6),score_threshold=0.90)))))

Suspicious frame: 01517.jpg, with 879 empty frames before and 569 empty frames after, info: {'xmin': 1611.41748046875, 'ymin': 14.920388221740723, 'xmax': 1636.544921875, 'ymax': 41.916534423828125, 'score': 0.9402928948402405}
Suspicious frame: 02087.jpg, with 1449 empty frames before and 5 empty frames after, info: {'xmin': 1643.8026123046875, 'ymin': 64.59578704833984, 'xmax': 1669.826171875, 'ymax': 90.81431579589844, 'score': 0.9194740056991577}
Suspicious frame: 02093.jpg, with 1455 empty frames before and 352 empty frames after, info: {'xmin': 1597.972412109375, 'ymin': 95.5059585571289, 'xmax': 1623.1439208984375, 'ymax': 121.66190338134766, 'score': 0.973003089427948}
Suspicious frame: 02446.jpg, with 1808 empty frames before and 8 empty frames after, info: {'xmin': 1850.289794921875, 'ymin': 588.3661499023438, 'xmax': 1876.76953125, 'ymax': 614.4552001953125, 'score': 0.9041323065757751}
Starting Post-Processing...
Index founded. Index Start:418, Index End:2494
Sequence: [150

In [32]:
gt_ann_file = Path.home() / PATH_XML_5
pred_file = Path.home() / f'{PATH_OUT}/ID_5_256942_252050.json'

evaluator = TrackingEvaluator(gt_ann_file, pred_file)
evaluator.load_data()
evaluator.compute_frame_indices()
evaluator.evaluate_metrics()
mse5 = evaluator.compute_mse(); 

GT   frames: 420 - 2580
PRED frames: 424 - 2966
Frame Index Range: (420, 2966)
Total frames: 346
Total predictions: 364
Matches: 276 (0.798)
No matches: 44 (0.127)
No predictions: 26 (0.075)
No frame data: 2201 (0.864)
Mean Squared Error: 0.02769434631639687


In [33]:
gt_ann_file = Path.home() / PATH_XML_6
pred_file = Path.home() / f'{PATH_OUT}/ID_6_256942_252050.json'

evaluator = TrackingEvaluator(gt_ann_file, pred_file)
evaluator.load_data()
evaluator.compute_frame_indices()
evaluator.evaluate_metrics()
mse6 = evaluator.compute_mse(); 

GT   frames: 417 - 2495
PRED frames: 418 - 2494
Frame Index Range: (417, 2495)
Total frames: 263
Total predictions: 284
Matches: 249 (0.947)
No matches: 12 (0.046)
No predictions: 2 (0.008)
No frame data: 1816 (0.873)
Mean Squared Error: 0.005526862526945486


In [34]:
print(f'FINAL MSE {np.mean([mse5, mse6])}')

FINAL MSE 0.016610604421671178


# CREAZIONE DEL VIDEO

In [35]:
import cv2
import json
import os
import re
import numpy as np

def crea_video_con_annotazioni(cartella_immagini, file_json, output_video):
    
    print(f"Caricamento del file JSON: {file_json}")
    
    with open(file_json, 'r') as f:
        annotazioni = json.load(f)
        
    print(f"Annotazioni caricate. Numero di frame nel JSON: {len(annotazioni)}")

    frame_files = sorted(
        [f for f in os.listdir(cartella_immagini) if f.endswith('.jpg')],
        key=lambda f: int(re.sub('\D', '', f)) #cosi otteniamo solo il numero
    )
    
    print(f"Numero di immagini trovate nella cartella: {len(frame_files)}")

    img = cv2.imread(os.path.join(cartella_immagini, frame_files[0]))
    altezza, larghezza, _ = img.shape
    print(f"Dimensioni del frame: larghezza = {larghezza}, altezza = {altezza}")

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter(output_video, fourcc, 25.0, (larghezza, altezza))
    print(f"Video writer creato. Output: {output_video}")

    # solo per i 5 frame precedenti
    traccia_palla = []

    for i, frame_file in enumerate(frame_files):
        print(f"Processando il frame: {frame_file}")

        frame_number = re.search(r'(\d+)', frame_file).group(0)
        frame_name = f"{int(frame_number):05d}.jpg"

        print(f"Nome frame formato per JSON: {frame_name}")

        frame = cv2.imread(os.path.join(cartella_immagini, frame_file))

        previsione = None
        if frame_name in annotazioni:
            x, y = annotazioni[frame_name]['x'], annotazioni[frame_name]['y']
            print(f"Annotazioni trovate per {frame_name}: x = {x}, y = {y}")

            if x != -1 and y != -1:
                print(f"Aggiungendo cerchio al frame {frame_name} alle coordinate ({x}, {y})")
                traccia_palla.append((int(x), int(y)))
                previsione = (int(x), int(y))
            else:
                # Se l'annotazione è (-1, -1),la dobbiamo comunque considerare altrimenti abbiamo un video sballato
                print(f"Frame {frame_name} con annotazione (-1,-1), aggiunto alla traccia ma senza cerchio.")
                traccia_palla.append((-1, -1))
        else:
            print(f"Nessuna annotazione trovata per il frame: {frame_name}")
            traccia_palla.append((-1, -1))

        # Limitiamo la traccia della palla ai 5 frame precedenti
        if len(traccia_palla) > 5:
            traccia_palla.pop(0)

        for idx, (px, py) in enumerate(traccia_palla[:-1]): 
            if px != -1 and py != -1:
                color = (0, 0, int(255 * (idx / len(traccia_palla))))  #Per ottenere un effetto graduale del colore rosso
                if idx > 0 and traccia_palla[idx - 1] != (-1, -1):
                    prev_x, prev_y = traccia_palla[idx - 1]
                    cv2.line(frame, (prev_x, prev_y), (px, py), color, 2)

        if previsione:
            px, py = previsione
            cv2.circle(frame, (px, py), 10, (0, 0, 255), 2)

        video.write(frame)

    video.release()
    print(f"Video salvato come: {output_video}")


In [36]:
cartella_immagini = '/kaggle/input/computer-vision-futbal/dataset/dataset/test/ID-5'
file_json = f"{PATH_OUT}/ID_5_256942_252050.json"
output_video = f"{PATH_OUT}/video_5.avi"

crea_video_con_annotazioni(cartella_immagini, file_json, output_video)

Caricamento del file JSON: /kaggle/working/ID_5_256942_252050.json
Annotazioni caricate. Numero di frame nel JSON: 2999
Numero di immagini trovate nella cartella: 2999
Dimensioni del frame: larghezza = 1920, altezza = 1080
Video writer creato. Output: /kaggle/working/video_5.avi
Processando il frame: frame_0.jpg
Nome frame formato per JSON: 00000.jpg
Annotazioni trovate per 00000.jpg: x = -1, y = -1
Frame 00000.jpg con annotazione (-1,-1), aggiunto alla traccia ma senza cerchio.
Processando il frame: frame_1.jpg
Nome frame formato per JSON: 00001.jpg
Annotazioni trovate per 00001.jpg: x = -1, y = -1
Frame 00001.jpg con annotazione (-1,-1), aggiunto alla traccia ma senza cerchio.
Processando il frame: frame_2.jpg
Nome frame formato per JSON: 00002.jpg
Annotazioni trovate per 00002.jpg: x = -1, y = -1
Frame 00002.jpg con annotazione (-1,-1), aggiunto alla traccia ma senza cerchio.
Processando il frame: frame_3.jpg
Nome frame formato per JSON: 00003.jpg
Annotazioni trovate per 00003.jpg: 

In [37]:
cartella_immagini = '/kaggle/input/computer-vision-futbal/dataset/dataset/test/ID-6'
file_json = f"{PATH_OUT}/ID_6_256942_252050.json"
output_video = f"{PATH_OUT}/video_6.avi"

crea_video_con_annotazioni(cartella_immagini, file_json, output_video)

Caricamento del file JSON: /kaggle/working/ID_6_256942_252050.json
Annotazioni caricate. Numero di frame nel JSON: 3000
Numero di immagini trovate nella cartella: 3000
Dimensioni del frame: larghezza = 1920, altezza = 1080
Video writer creato. Output: /kaggle/working/video_6.avi
Processando il frame: frame_0.jpg
Nome frame formato per JSON: 00000.jpg
Annotazioni trovate per 00000.jpg: x = -1, y = -1
Frame 00000.jpg con annotazione (-1,-1), aggiunto alla traccia ma senza cerchio.
Processando il frame: frame_1.jpg
Nome frame formato per JSON: 00001.jpg
Annotazioni trovate per 00001.jpg: x = -1, y = -1
Frame 00001.jpg con annotazione (-1,-1), aggiunto alla traccia ma senza cerchio.
Processando il frame: frame_2.jpg
Nome frame formato per JSON: 00002.jpg
Annotazioni trovate per 00002.jpg: x = -1, y = -1
Frame 00002.jpg con annotazione (-1,-1), aggiunto alla traccia ma senza cerchio.
Processando il frame: frame_3.jpg
Nome frame formato per JSON: 00003.jpg
Annotazioni trovate per 00003.jpg: 